<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Build Retro-DB and Retro-DB-light databases

_by Grayson Badgley and Joe Hamman (CarbonPlan), December 17, 2020_

This notebook munges a collection of datasets to create what we refer to as the `Retro-DB` and
`Retro-DB-light` databases.

The source datasets include:

- Project-DB: A Google sheet including the digitized forest carbon offset project details.
- ARB Issuance table: A spreadsheet including the official ARBOC issuances (version dated September
  9, 2020).
- MTBS Fire Risk: GeoJSON datasets including aggregated by supersection and ecoregion.
- Project shapes: A collection of GeoJSON datasets describing the boundaries of each project in
  Project-DB.

In addition to the static datasets above, we derive the following datasets:

- OPDR-Calculated: issuance derived from IFM-1, IFM-3, IFM-7, IFM-8, and secondary effects (SE).

Like all munging workflows, not everything here is perfectly polished, but the resulting datasets
are clean and should be useful for holistic analysis of the programs included.

At the end of this notebook, we end up with three "allocation" values:

- Issuance: the official, issued allocation of ARBOCs as recorded by ARB.
- OPDR-Reported: the OPO/APD reported ARBC
- OPDR-Calculated: issuance derived from IFM-1, IFM-3, IFM-7, IFM-8, and secondary effects (SE).

In a separate notebook, we explore cases where these values diverge.


In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

import os
import json
import pathlib
import sys

from itertools import permutations
import geopandas
import numpy as np
import pandas as pd
from shapely.geometry import Point

from carbonplan_retro.load.project_db import load_project_db
from carbonplan_retro.load.issuance import load_issuance_table
from carbonplan_retro.analysis import allocation

# options
EXCLUDE_GRADUATED_PROJECTS = True
WRITE_SHAPES = False

# paths
DROPBOX = '/Users/jhamman/CarbonPlan Dropbox/Projects/Microsoft/Forests-Retrospective'
BUCKET = f'{DROPBOX}/carbonplan-retro'
VERSION = 'v1.0'

## Load retro-db and issuance table


In [ ]:
project_db = load_project_db("Forest-Offset-Projects-v0.3", use_cache=False, save=False)

There is this strange sub-class of projects within the compliance market which we refer to as
"graduated" projects -- these are projects that started out in the "Early Action" period and later
"graduated" into the full-fledged compliance program. Unfortunately, these projects tend to be
materially deficient and tough to work with -- there are all sorts of issues with getting their
numbers correct.


In [ ]:
if EXCLUDE_GRADUATED_PROJECTS:
    project_db = project_db[~project_db["project"]["early_action"].str.startswith("CAR")]

We have also downloaded the spatial boundaries for each project. Here we simply standardize the
GeoJSON format.


In [ ]:
if WRITE_SHAPES:
    for i, proj in enumerate(project_db.index):
        src = f"{DROPBOX}/shapes/{proj}.json"
        dst = f"{BUCKET}/projects/{proj}"
        dst_file = os.path.join(dst, "shape.json")
        os.makedirs(dst, exist_ok=True)

        with open(src) as f:
            data = json.load(f)

        if len(data["features"]) == 1:
            data["features"][0]["properties"] = {"id": proj}
        else:
            gdf = geopandas.GeoDataFrame.from_file(src)
            data = json.loads(geopandas.GeoDataFrame(geometry=[gdf.unary_union]).to_json())
            data["features"][0]["properties"] = {"id": proj}

        with open(dst_file, "w") as f:
            json.dump(data, f, indent=2, allow_nan=False)

## Get Issuance


In [ ]:
# TODO: store this issuance file somewhere else!
issuance_table = load_issuance_table(
    f"{DROPBOX}/documents-of-interest/arb/issuance/arboc_issuance_2020-09-09.xlsx"
)

# One project has multiple issuance events in its first reporting period, aggregate them
agg_by_rp = issuance_table.groupby(["opr_id", "arb_rp_id"])[["allocation"]].sum()
issuance_first_rp = agg_by_rp.xs("A", level=1)["allocation"]
issuance_first_rp.head()

## OPDR-calculated

OPDRs report five individual components that we use to recalculate ARBOC issuance:

- IFM-1: standing live
- IFM-3: standing dead
- IFM-7: in-use wood products
- IFM-8: landfilled wood products
- Secondary Effects: market leakage &etc.

We use these five "components", as reported in the OPDR for both the Baseline scenario
(imaginary/counterfactual) and the Project scenario (what actually happened), to re-derive the ARBOC
allocation. This step (i) gives us confidence in the integrity of our data entry and (ii) lays the
foundation for _re-calculating_ ARBOCs under different common practice scenarios (see Notebook TK).


In [ ]:
opdr_calculated = allocation.calculate_allocation(project_db, round_intermediates=False)

opdr_calculated = opdr_calculated
opdr_calculated.head()

## OPDR-Reported


In [ ]:
reported = project_db[("rp_1", "allocation", "")]
reported.head()

Now that we have three Series and can populate the `rp_1.allocation` fields:


In [ ]:
project_db[("rp_1", "allocation", "reported")] = reported
project_db[("rp_1", "allocation", "calculated")] = opdr_calculated
project_db[("rp_1", "allocation", "issuance")] = issuance_first_rp

## Project geometries

Here we extract the centroid of each project from the project geometries.


In [ ]:
# add project centroids from shapefiles to a new column
coords = [list([]) for i in range(len(project_db))]

for i, proj in enumerate(project_db.index):
    gdf = geopandas.GeoDataFrame.from_file(f"{BUCKET}/projects/{proj}/shape.json")
    coords[i] = [gdf.geometry.item().centroid.x, gdf.geometry.item().centroid.y]
project_db[("project", "shape_centroid", "")] = coords

## Fire risk

In this step, we extract the integrated fire risk for each project based on analysis done
aggregating historical fire by supersection and Bailey's ecoregions.


In [ ]:
# TODO: use project boundaries and weight by overlap with each polygon.
for key in ["supersections", "baileys"]:
    gdf = geopandas.GeoDataFrame.from_file(f"{BUCKET}/fire/{key}.json")

    fire_risk = []
    for i, (proj, row) in enumerate(project_db.iterrows()):
        lon, lat = row[("project", "shape_centroid", "")]
        p = Point(lon, lat)
        try:
            r = -9999
            for j in gdf.index:
                if gdf.geometry[j].contains(p):
                    r = gdf["integrated_risk"][j]
                    break
            fire_risk.append(r)
        except:
            fire_risk.append(-9999)

    project_db[("project", "mtbs_fire_risk", key)] = pd.Series(fire_risk, index=project_db.index)

## Make Retro-DB light

The schema here is istill in flux. Best to just look at the code below for details.


In [ ]:
def make_rp_1(row):
    d = {}
    try:
        d["start_date"] = row["rp_1", "start", ""].strftime("%Y-%m-%d")
    except ValueError:
        d["start_date"] = None

    try:
        d["end_date"] = row["rp_1", "end", ""].strftime("%Y-%m-%d")
    except ValueError:
        d["end_date"] = None

    d.update(row["rp_1"]["components"][["ifm_1", "ifm_3", "ifm_7", "ifm_8"]].to_dict())
    d["secondary_effects"] = row["rp_1", "secondary_effects", ""]
    d["confidence_deduction"] = row["rp_1", "confidence_deduction", ""]

    return d


def valid_species(species):

    for c in species:
        if "basal_area" not in c:
            print("setting missing basal_area to None")
            c["basal_area"] = None
        if "fraction" not in c:
            print("setting missing fraction to None")
            c["fraction"] = None
    return species


def make_supersections(row):
    species = row["project", "species", ""]
    if not isinstance(species, dict):
        print("species was not a dict")
        species = None
    else:
        species = species.copy()

    data = row["project", "assessment_areas", ""]

    if not data:
        print("assessment_areas was empty or null")
        return []
    else:
        data = data.copy()

    for aa in data:
        if species:
            if str(aa["code"]) in species:
                aa["species"] = valid_species(species[str(aa["code"])])
            elif "all" in species:
                aa["species"] = []
            else:
                print("did not find species key %s" % aa["code"])
                aa["species"] = []

        else:
            aa["species"] = []
    if species and "all" in species:
        print("all field was used")
        data.append({"code": 999, "species": species["all"]})

    return data


make_supersections(project_db.loc["CAR1094"])

In [ ]:
def make_retro_db_light(project_db):

    projects = []

    for i, row in project_db.iterrows():
        print(i)
        p = {
            "id": row[("project", "opr_id", "")],
            "opr_id": row[("project", "opr_id", "")],
            "arb_id": row[("project", "arb_id", "")],
            "name": row[("project", "name", "")],
            "apd": row[("project", "apd", "")],
            "opo": row[("project", "opo", "")],
            "owners": row[("project", "owners", "")],
            "developers": row[("project", "developers", "")],
            "attestor": row[("rp_1", "attestation", "name")],
            "is_opo": row[("rp_1", "attestation", "is_opo")],
            "coordinates": row[("project", "coordinates", "")],
            "shape_centroid": row[("project", "shape_centroid", "")],
            "supersection_ids": row[("project", "supersection_ids", "")],
            "acreage": row[("project", "acreage", "")],
            "buffer_contribution": row[("rp_1", "buffer_contribution", "")],
            "arbocs": {
                "issuance": row[("rp_1", "allocation", "issuance")],
                "calculated": row[("rp_1", "allocation", "calculated")],
                "reported": row[("rp_1", "allocation", "reported")],
            },
            "carbon": {
                "initial_carbon_stock": {
                    "value": row[("baseline", "initial_carbon_stock", "")],
                    "units": "tCO2e-1",
                },
                "common_practice": {
                    "value": row[("baseline", "common_practice", "")],
                    "units": "tCO2e-1",
                },
            },
            "baseline": row["baseline"]["components"][
                ["ifm_1", "ifm_3", "ifm_7", "ifm_8"]
            ].to_dict(),
            "rp_1": make_rp_1(row),
            "assessment_areas": make_supersections(row),
            "permanence": {
                "arb_total_risk": row[("project", "reversal_risk", "")],
                "arb_fire_risk": row[("project", "fire_risk", "")],
                "mtbs_fire_risk_supersection": row[("project", "mtbs_fire_risk", "supersections")],
                "mtbs_fire_risk_baileys": row[("project", "mtbs_fire_risk", "baileys")],
            },
            "notes": row["project", "notes", ""],  # include or not?
            "comment": "",  # get from google sheet
        }

        for k in ["coordinates", "shape_centroid"]:
            if not p[k]:
                p[k] = []
            else:
                if p[k][0] == -9999:
                    p[k] = []

        projects.append(p)

    return projects


retro_db_light = make_retro_db_light(project_db)
retro_db_light[:2]

write to JSON:


In [ ]:
def write_projects(project_collection, output):
    with open(output, "w") as outfile:
        json.dump(project_collection, outfile, indent=2)


write_projects(retro_db_light, f"{BUCKET}/projects/retro-db-light-{VERSION}.json")

write to CSV:


In [ ]:
df = pd.DataFrame(retro_db_light)
df.head()

dict_cols = ["arbocs", "carbon", "permanence"]

for col_key in dict_cols:
    for field_key in df.iloc[0][col_key].keys():
        new_key = f"{col_key}_{field_key}"
        vals = [row[field_key] for row in df[col_key]]
        if isinstance(vals[0], dict):
            vals = [v["value"] for v in vals]
        df[new_key] = vals
df = df.drop(columns=dict_cols)

df.to_csv(f"{BUCKET}/projects/retro-db-light-{VERSION}.csv")